# $U(1)$ Gauge Model using L2HMC in graph mode

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib notebook

from l2hmc_eager import gauge_dynamics_eager as gde

Using TensorFlow backend.


In [2]:
#tf.enable_eager_execution()
tfe = tf.contrib.eager

from u1_model_eager import GaugeModelEager, train_one_iter
from gauge_model import GaugeModel

import utils.gauge_model_helpers as helpers

In [3]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [5]:
params = {
    'time_size': 16,
    'space_size': 16,
    'link_type': 'U1',
    'dim': 2,
    'beta': 8.,
    'num_samples': 2,
    'num_steps': 10,
    'eps': 0.1,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.96,
    'train_steps': 1000,
    'record_loss_every': 50,
    'data_steps': 100,
    'save_steps': 500,
    'print_steps': 5,
    'logging_steps': 100,
    'clip_value': 50,
    'rand': False,
    'metric': 'l2',
}
tf.reset_default_graph()

## Create model

In [13]:
del model

In [6]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [15]:
tf.reset_default_graph()

In [ ]:
model = GaugeModel(params=params,
                   config=config,
                   sess=None,
                   conv_net=True,
                   hmc=False,
                   log_dir=None,
                   restore=False,
                   eps_trainable=True)

In [ ]:
model.train(1000, kill_sess=False)

--------------------------------------------------------------------------
    STEP         LOSS     TIME/STEP    ACCEPT %      EPS          LR     
--------------------------------------------------------------------------


In [8]:
samples_history = model.run(500)

Running (trained) L2HMC sampler for 500 steps...
step:   0     model invariant time / step: 166.1 
step:   1     model invariant time / step: 0.1795
step:   2     model invariant time / step: 0.219 
step:   3     model invariant time / step: 0.1925
step:   4     model invariant time / step: 0.1951
step:   5     model invariant time / step: 0.1738
step:   6     model invariant time / step: 0.2084
step:   7     model invariant time / step: 0.2007
step:   8     model invariant time / step: 0.211 
step:   9     model invariant time / step: 0.1934
step:   10    model invariant time / step: 0.2049
step:   11    model invariant time / step: 0.2117
step:   12    model invariant time / step: 0.2033
step:   13    model invariant time / step: 0.1854
step:   14    model invariant time / step: 0.2134
step:   15    model invariant time / step: 0.1905
step:   16    model invariant time / step: 0.185 
step:   17    model invariant time / step:  0.17 
step:   18    model invariant time / step: 0.1804
s

step:  163    model invariant time / step: 0.392 
step:  164    model invariant time / step: 0.4771
step:  165    model invariant time / step: 0.2769
step:  166    model invariant time / step: 0.2896
step:  167    model invariant time / step: 0.2908
step:  168    model invariant time / step: 0.3737
step:  169    model invariant time / step: 0.3132
step:  170    model invariant time / step: 0.2379
step:  171    model invariant time / step: 0.3232
step:  172    model invariant time / step: 0.3206
step:  173    model invariant time / step: 0.2062
step:  174    model invariant time / step: 0.2188
step:  175    model invariant time / step: 0.1972
step:  176    model invariant time / step: 0.2066
step:  177    model invariant time / step: 0.1928
step:  178    model invariant time / step: 0.2006
step:  179    model invariant time / step: 0.1952
step:  180    model invariant time / step: 0.2077
step:  181    model invariant time / step: 0.3187
step:  182    model invariant time / step: 0.2013


step:  328    model invariant time / step: 0.4345
step:  329    model invariant time / step: 0.2934
step:  330    model invariant time / step: 0.2899
step:  331    model invariant time / step: 0.1804
step:  332    model invariant time / step: 0.1788
step:  333    model invariant time / step: 0.1789
step:  334    model invariant time / step: 0.1824
step:  335    model invariant time / step: 0.2142
step:  336    model invariant time / step: 0.341 
step:  337    model invariant time / step: 0.2545
step:  338    model invariant time / step: 0.2831
step:  339    model invariant time / step: 0.2398
step:  340    model invariant time / step: 0.2022
step:  341    model invariant time / step: 0.2001
step:  342    model invariant time / step: 0.2136
step:  343    model invariant time / step: 0.199 
step:  344    model invariant time / step: 0.2385
step:  345    model invariant time / step: 0.1842
step:  346    model invariant time / step: 0.198 
step:  347    model invariant time / step: 0.1963


step:  492    model invariant time / step: 0.2814
step:  493    model invariant time / step: 0.274 
step:  494    model invariant time / step: 0.237 
step:  495    model invariant time / step: 0.261 
step:  496    model invariant time / step: 0.2162
step:  497    model invariant time / step: 0.1792
step:  498    model invariant time / step: 0.1974
step:  499    model invariant time / step: 0.1811
done. Samples saved to: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_60/run_info/samples_history.pkl.


In [18]:
model.dynamics.position_fn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_x1 (Conv2D)             multiple                  608       
_________________________________________________________________
conv_v1 (Conv2D)             multiple                  608       
_________________________________________________________________
max_pool_x1 (MaxPooling2D)   multiple                  0         
_________________________________________________________________
max_pool_v1 (MaxPooling2D)   multiple                  0         
_________________________________________________________________
conv_x2 (Conv2D)             multiple                  8256      
_________________________________________________________________
conv_v2 (Conv2D)             multiple                  8256      
_________________________________________________________________
max_pool_x2 (MaxPooling2D)   multiple                  0         
__________

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)